In [29]:
from pathlib import Path
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
from cryptography import x509
import re

In [30]:
def egcd(a, b):
    """Algorithme d'Euclide étendu.
    Retourne (g, x, y) tels que g = gcd(a, b) et ax + by = g.
    """
    if b == 0:
        return a, 1, 0
    g, x1, y1 = egcd(b, a % b)
    return g, y1, x1 - (a // b) * y1

In [31]:
def inv_mod(a, n):
    """Inverse modulaire : trouve x tel que a*x ≡ 1 mod n."""
    g, x, y = egcd(a, n)
    if g != 1:
        raise ValueError("Pas d'inverse modulaire")
    return x % n

In [32]:
def compute_u_i(N1, N2, N3):
    N = N1 * N2 * N3

    N_hat1 = N // N1
    N_hat2 = N // N2
    N_hat3 = N // N3

    u1 = inv_mod(N_hat1 % N1, N1)
    u2 = inv_mod(N_hat2 % N2, N2)
    u3 = inv_mod(N_hat3 % N3, N3)

    return (u1, u2, u3), (N_hat1, N_hat2, N_hat3)


In [33]:
def compute_m_cubed(c1, c2, c3, N1, N2, N3):
    (u1, u2, u3), (Nh1, Nh2, Nh3) = compute_u_i(N1, N2, N3)
    N = N1 * N2 * N3

    m3 = (c1 * u1 * Nh1 +
          c2 * u2 * Nh2 +
          c3 * u3 * Nh3) % N

    return m3, N


In [34]:
def integer_cube_root(n):
    """Renvoie la racine cubique entière si elle existe exactement."""
    x = int(round(n ** (1/3)))
    while (x + 1)**3 <= n:
        x += 1
    while x**3 > n:
        x -= 1
    if x**3 != n:
        raise ValueError("La racine cubique n'est pas un entier parfait.")
    return x


In [35]:
def recover_message_from_broadcast(c1, c2, c3, N1, N2, N3):
    m3, N = compute_m_cubed(c1, c2, c3, N1, N2, N3)
    m = integer_cube_root(m3)
    return m


In [36]:
import base64
from pathlib import Path

def read_cipher_as_int(filename):
    data_b64 = Path(filename).read_bytes().strip()
    data_bytes = base64.b64decode(data_b64)
    return int.from_bytes(data_bytes, "big")

c0 = read_cipher_as_int("c0")
c1 = read_cipher_as_int("c1")
c2 = read_cipher_as_int("c2")


In [38]:
def read_modulus_from_file(filename: str):
    """
    Lit un fichier de clé/certificat RSA (PEM, DER ou format texte simple)
    et renvoie (N, e).

    Stratégie :
      1) essayer PEM/DER standard (clé publique ou certificat)
      2) sinon, lire comme texte et chercher un gros entier (décimal ou hex)
    """
    data = Path(filename).read_bytes()

    # ---------- 1) Essais formats binaires/PEM "classiques" ----------
    # PEM certificat
    if b"BEGIN CERTIFICATE" in data:
        try:
            cert = x509.load_pem_x509_certificate(data, backend=default_backend())
            pubkey = cert.public_key()
            numbers = pubkey.public_numbers()
            return numbers.n, numbers.e
        except Exception:
            pass

    # PEM clé publique
    if b"BEGIN PUBLIC KEY" in data or b"BEGIN RSA PUBLIC KEY" in data:
        try:
            pubkey = serialization.load_pem_public_key(data, backend=default_backend())
            numbers = pubkey.public_numbers()
            return numbers.n, numbers.e
        except Exception:
            pass

    # DER clé publique ou DER cert
    try:
        pubkey = serialization.load_der_public_key(data, backend=default_backend())
        numbers = pubkey.public_numbers()
        return numbers.n, numbers.e
    except Exception:
        try:
            cert = x509.load_der_x509_certificate(data, backend=default_backend())
            pubkey = cert.public_key()
            numbers = pubkey.public_numbers()
            return numbers.n, numbers.e
        except Exception:
            pass

    # ---------- 2) Fallback : analyse texte ----------
    text = data.decode(errors="ignore")

    # 2a : chercher "N = 123456789..."
    m_dec = re.search(r'N\\s*=?\\s*([0-9]{10,})', text)
    if m_dec:
        return int(m_dec.group(1)), 3

    # 2b : détecter ligne contenant un grand entier décimal
    for line in text.splitlines():
        line = line.strip()
        if re.fullmatch(r"[0-9]{10,}", line):
            return int(line), 3

    # 2c : détecter grand bloc hexadécimal
    hex_candidates = re.findall(r"[0-9A-Fa-f]{64,}", text)
    if hex_candidates:
        return int(hex_candidates[0], 16), 3

    raise ValueError(f"Format de clé non reconnu pour {filename}")

In [39]:
N0, e0 = read_modulus_from_file("clef0_pub.pem")
N1, e1 = read_modulus_from_file("clef1_pub.pem")
N2, e2 = read_modulus_from_file("clef2_pub.pem")

print(N0, e0)
print(N1, e1)
print(N2, e2)

ValueError: Format de clé non reconnu pour clef1_pub.pem

In [ ]:
m = recover_message_from_broadcast(c0, c1, c2, N0, N1, N2)
print("m =", m)


In [ ]:
m_bytes = m.to_bytes((m.bit_length() + 7) // 8, "big")
print(m_bytes)

try:
    print(m_bytes.decode("utf-8"))
except:
    print("Décodage impossible (pas UTF-8).")
